In [1]:
import pandas as pd
import numpy as np
import csv

In [2]:
anime_data = pd.read_csv('dataset/anime.csv')
anime_data = anime_data[['MAL_ID', 'Name','Genres','Completed']]
anime_data

,MAL_ID,Name,Genres,Completed
0,1,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",718161
1,5,Cowboy Bebop: Tengoku no Tobira,"Action, Drama, Mystery, Sci-Fi, Space",208333
2,6,Trigun,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",343492
3,7,Witch Hunter Robin,"Action, Mystery, Police, Supernatural, Drama, ...",46165
4,8,Bouken Ou Beet,"Adventure, Fantasy, Shounen, Supernatural",7314
...,...,...,...,...
17557,48481,Daomu Biji Zhi Qinling Shen Shu,"Adventure, Mystery, Supernatural",0
17558,48483,Mieruko-chan,"Comedy, Horror, Supernatural",0
17559,48488,Higurashi no Naku Koro ni Sotsu,"Mystery, Dementia, Horror, Psychological, Supe...",0
17560,48491,Yama no Susume: Next Summit,"Adventure, Slice of Life, Comedy",2


In [3]:
# Key: MAL_ID, Value: [genre1, genre2,...]
anime_dict = dict()

anime_genre = dict()

for index, row in anime_data.iterrows():
    mal_id = row[0]
    genres_list = row[2].split(", ")
    
    for genre in genres_list:
        anime_genre[genre] = 1
    anime_dict[mal_id] = genres_list

total_genres = []
for key in anime_genre.keys():
    total_genres.append(key)

total_genres.sort()

In [4]:
top_users = 2000
top_animes = 200

In [5]:
user_rating = pd.read_csv('rating_complete.csv')
ranking = user_rating.groupby(['user_id']).count().sort_values('anime_id',ascending=False)
ranking = ranking[0:top_users]
# key: user_id, value:{mal_id:score}
user_anime_dict = dict()

In [6]:
def caluculate_genre_avg_score_for_user(user_id):
    # Key: genre, Value: count of this genre
    genre_count = dict()
    # Key: genre, Value: score of this genre
    genre_score = dict()
    
    current_user = dict()

    
    for genre in total_genres:
        genre_count[genre] = 0
        genre_score[genre] = 0
    
    piece = user_rating.loc[user_rating['user_id'] == user_id]
    
    for index, row in piece.iterrows():
        mal_id = row["anime_id"]
        score = row["rating"]
        
        current_user[mal_id] = score
        
        mal_genre = anime_dict[mal_id]
        
        for g in mal_genre:
            genre_count[g] = genre_count[g] + 1
            genre_score[g] = genre_score[g] + score
    
    result = []
    for genre in total_genres:
        if genre_count[genre] == 0:
            result.append(0)
            continue
        result.append(genre_score[genre] / genre_count[genre])
    
    user_anime_dict[user_id] = current_user
    
    return result
    

In [7]:
# Key: user id, value: avg scores, order the same as total_genres
user_dict = dict()

for index, row in ranking.iterrows():
    user_id  = index
    
    result = caluculate_genre_avg_score_for_user(user_id)
    user_dict[user_id] = result

In [8]:
path = "dataset/users_genres_rating.csv"
with open(path, "w") as f:
    cw = csv.writer(f)
    header = ["user_id"] + total_genres
    cw.writerow(header)
    
    for user_id in user_dict.keys():
        cw.writerow([user_id] + user_dict[user_id])
    
    f.close()
    

In [9]:
def get_user_completed_anime(mal_id):
    result = []
    
    for i, r in ranking.iterrows():
        uid = i
        
        if mal_id in user_anime_dict[uid]:
            result.append(str(uid) + "|" + str(user_anime_dict[uid][mal_id]))
            
    return result

In [10]:
# Key: mal_id, Value: list of (user id|score)
anime_user = dict()


animes = anime_data.sort_values('Completed', ascending=False)[0:top_animes]
for index, row in animes.iterrows():
    
    mal_id = row["MAL_ID"]
    
    result = get_user_completed_anime(mal_id)
    
    anime_user[mal_id] = result

In [11]:
path = "dataset/top_animes_complete.csv"
with open(path, "w") as f:
    cw = csv.writer(f)
    header = ["MAL_ID", "completed_user_ids"]
    cw.writerow(header)
    
    for mal_id in anime_user.keys():
        cw.writerow([mal_id, ", ".join(anime_user[mal_id])])
    
    f.close()